In [1]:
api_key = "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"

In [2]:
from ma_finkg import MA_FinKG

# Set OpenRouter API key and model  
openrouter_key = api_key
model = "openai/gpt-4.1-mini"

# Initialize with explicit ontology choice (refind)
kg = MA_FinKG(model=model, api_key=openrouter_key) # fire, nyt11, conllpp, refind - if not specified defaults to "default"

In [5]:
# Sentence FIRE Example - Longer Text from Nvidia Article
sent = "Mr. Rooke owns and manages Rooke Fiduciary Management , a private trust company , which specializes in the investment management of publicly held securities and the oversight of a multitude of trust investments ."
text = """
Nvidia, the world’s most valuable company, reported another quarter of sustained sales growth in its earnings statement Wednesday, with $46.7 billion in revenue, a 56% increase compared to the same period last year. That growth was largely fueled by AI-dominated data center business, which saw a 56% year-over-year increase in revenue.
Nvidia also saw its net income grow substantially since last year. The company reported a net income of $26.4 billion in the second quarter, a 59% spike since the same period last year. 
All told, the company brought in $41.1 billion in revenue from data center sales in the quarter, suggesting that AI companies’ demand for cutting-edge GPUs continues to grow. The company’s most advanced generation of chips, Blackwell, accounted for $27 billion of those sales.
“Blackwell is the AI platform the world has been waiting for,” said CEO Jensen Huang in a statement accompanying the release. “The AI race is on, and Blackwell is the platform at its center.”
The company made particular note of its role in the launch of OpenAI’s open source gpt-oss models earlier this month, which involved processing “1.5 million tokens per second on a single Nvidia Blackwell GB200 NVL72 rack-scale system.”
The earnings also gave a look at Nvidia’s ongoing struggle to sell its chips in Chinese markets. The company reported no sales of its China-focused H20 chip to Chinese customers in the past quarter; Nvidia did report $650 million worth of H20 chips had been sold to a customer outside China.
The United States has long restricted sales of advanced GPUs to Chinese customers — but the geopolitical situation has changed significantly under President Trump. The company is now permitted to sell chips to China as long as it pays a 15% export tax to the U.S. Treasury, as a result of an unconventional arrangement that legal scholars have described as an unconstitutional abuse of power. 
"""
const = kg.generate(text)


[0.0s] Starting knowledge graph construction...
[1.0s] Creating ontology...

[ONTOLOGY] Creating dynamic ontology for text: 1916 chars
[3.3s] Creating ontology... done                                                                   
[ONTOLOGY] Dynamic ontology creation succeeded

[ONTOLOGY] Entity Types: ['Company', 'Revenue', 'Income', 'Chips', 'System', 'Country', 'President', 'Tax']
[ONTOLOGY] Relation Types: ['Reported', 'Saw', 'Accounted for', 'Role in', 'Struggle to sell', 'Restricted sales to', 'Permitted to sell', 'Described as']
[4.1s] Extracting entities and relations...
[10.3s] NER completed: 15 entities                                                                  
[21.4s] RE completed: 3 filtered triples                                                            

[REVISION] Validated: 15/15 entities, 3/3 triples
[22.2s] Finalizing results...
[22.2s] Construction completed!
[FINAL] Processing results from finalize
[FINAL] Knowledge graph ready: 15 entities, 3 triples


In [12]:
# Extracted Entities & Relations
nodes = const.entities
edges = const.triples
print(nodes)
print(edges)

[{'text': 'Nvidia', 'type': 'Company'}, {'text': 'OpenAI', 'type': 'Company'}, {'text': '$46.7 billion in revenue', 'type': 'Revenue'}, {'text': 'revenue', 'type': 'Revenue'}, {'text': '$41.1 billion in revenue', 'type': 'Revenue'}, {'text': 'net income', 'type': 'Income'}, {'text': 'net income of $26.4 billion', 'type': 'Income'}, {'text': 'Blackwell', 'type': 'Chips'}, {'text': 'H20 chip', 'type': 'Chips'}, {'text': 'H20 chips', 'type': 'Chips'}, {'text': 'Nvidia Blackwell GB200 NVL72 rack-scale system', 'type': 'System'}, {'text': 'China', 'type': 'Country'}, {'text': 'United States', 'type': 'Country'}, {'text': 'President Trump', 'type': 'President'}, {'text': '15% export tax', 'type': 'Tax'}]
[{'head': 'Nvidia', 'relation': 'Saw', 'tail': 'OpenAI'}, {'head': 'Nvidia', 'relation': 'Role in', 'tail': 'OpenAI'}, {'head': 'Nvidia', 'relation': 'Permitted to sell', 'tail': 'China'}]


In [17]:
import networkx as nx
from ipysigma import Sigma

G = nx.DiGraph()
for entity in nodes:
    text = entity['text']
    etype = entity['type']
    G.add_node(text, etype=etype, label=text)

for triple in edges:
    head = triple['head']
    relation = triple['relation']
    tail = triple['tail']

    if head not in G:
        G.add_node(head, etype="Value", label=head)
    if tail not in G:
        G.add_node(tail, etype="Value", label=tail)
    G.add_edge(head, tail, relation=relation, label=relation)

# Check graph properties
print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
print("Sample nodes:", list(G.nodes())[:10])
print("Sample edges:", list(G.edges(data=True))[:10])

Nodes: 15, Edges: 2
Sample nodes: ['Nvidia', 'OpenAI', '$46.7 billion in revenue', 'revenue', '$41.1 billion in revenue', 'net income', 'net income of $26.4 billion', 'Blackwell', 'H20 chip', 'H20 chips']
Sample edges: [('Nvidia', 'OpenAI', {'relation': 'Role in', 'label': 'Role in'}), ('Nvidia', 'China', {'relation': 'Permitted to sell', 'label': 'Permitted to sell'})]


In [16]:
# Visualize KG Graph

Sigma(
    G,
    node_label="label",
    node_color="etype",
    edge_label="label"
)

Sigma(nx.DiGraph with 15 nodes and 2 edges)